# Imports

In [1]:
using LinearAlgebra
using SparseArrays
using Distributions
using PyCall
using Statistics
using BenchmarkTools

@pyimport numpy 

# GMRES

In [105]:
function gmres(A, B, X0, max_iter)
    r = B - A * X0
    x = NaN
    q = Array{Any}(zeros(max_iter)) #array any, gdyż przechowuje i wektory i pojedyncze liczby 
    q[1] = r/norm(r)
    h = zeros((max_iter + 1, max_iter))
    
    #for m=1:max_iter
    for m=1:min(max_iter, size(A, 1))
        y = A * q[m]
        
        #for j=1:m-1
        for j=1:m
            h[j, m] = (q[j]' * y)[1]
            y = y - h[j, m] * q[j] 
        end
        
        h[m + 1, m] = norm(y)
        
        if (h[m + 1, m] != 0) && (m != max_iter)
            q[m + 1] = y / h[m + 1, m]
        end

        b = zeros(max_iter + 1)
        b[1] = norm(r)
        result = (h \ b)

        if m > max_iter - 2
            z = reduce(hcat, q[:])
        else
            z = q
        end
        
        x = numpy.dot(z, result)+X0
        
    end
    
    return x
end

gmres (generic function with 3 methods)

# Restarted GMRES

In [106]:
function restarted_gmres(A, B, X0, max_iter, res)
    new_x0 = gmres(A, B, X0, res)
    for i=2*res:res:max_iter
        new_x0 = gmres(A, B, new_x0, res)
    end
    return new_x0
end

restarted_gmres (generic function with 2 methods)

# BICGSTAB

In [26]:
function bicgstab(A, B, init_guess, iterations, accuracy)
    x_prev = init_guess
    r_prev = B - A*x_prev
    r0_arb = r_prev
    rho_prev = alpha = omega_prev = 1.0
    v_prev = p_prev = rho = 0.0
    result = NaN
    for iter = 1:iterations
        rho = dot(r0_arb, r_prev)
        beta = (rho/rho_prev) * (alpha/omega_prev)
        p = r_prev .+ (beta* (p_prev - omega_prev * v_prev))
        v = A * p
        alpha = rho'/(dot(r0_arb, v))
        h = x_prev + alpha*p
        s = r_prev - alpha*v
        if(norm(s) < accuracy)
            result = h
            break
        end
        t = A * s
        omega = dot(t, s) / dot(t, t)
        x = h + omega*s
        r = s - omega*t
        if(norm(r) < accuracy)
            result = x
            break
        end
        r_prev = r
        x_prev = x
        p_prev = p
        v_prev = v
        omega_prev = omega
        rho_prev = rho
    end
    return result
end

bicgstab (generic function with 1 method)

In [114]:
#generacja macierzy A oraz wektorów B i X0
sizee = 200
A = Symmetric(sprand(sizee,sizee,0.5))
B = rand(Uniform(0,1), sizee, 1)
X0 = rand(Uniform(0,1), sizee, 1)

200×1 Matrix{Float64}:
 0.19327237733312264
 0.9622757584734905
 0.6820618916367365
 0.8604539566607095
 0.45196051381025293
 0.03304491012672628
 0.36054911622040486
 0.4619360531758918
 0.16973340366602474
 0.7749697216800286
 0.2841398322034647
 0.8002293356941543
 0.698832616200092
 ⋮
 0.2366238337691684
 0.16797819768652533
 0.6457323934950034
 0.26795140947849716
 0.3167249758141737
 0.5503644604405664
 0.5387498523355869
 0.3237331069532885
 0.07166677396785093
 0.012073813159654723
 0.6642974944794033
 0.09602991319966403

In [123]:
#gmres
max_iter = 500
res = 150

#bicgstab
accuracy = 1e-10

1.0e-10

In [116]:
x3 = A\B

200×1 Matrix{Float64}:
 -0.4905537058445198
 -0.41326485921487893
 -0.33194836885398316
 -0.3258009107818391
  0.4761335071870722
 -0.2076850788308544
  0.5692231259136883
  0.9627746663812078
  0.009428632682884672
 -0.11719633849673634
 -0.8772739856669994
  0.16929144553006728
 -0.26149072780023463
  ⋮
  0.10653880994739161
  0.2391583978935533
 -0.08749999364363124
 -0.43512886247629834
 -0.657780057302917
  0.033482459197242916
 -0.3229007365240274
  0.3949663068216457
 -0.46783512441896796
 -0.22909727024877657
 -0.21348027521689547
  0.544816843441575

In [124]:
x2 = restarted_gmres(A, B, X0, 10*max_iter, res)
#x2 = gmres(A, B, X0, max_iter, accuracy, x3)

200×1 Matrix{Float64}:
 -0.4905537058445191
 -0.41326485921487666
 -0.33194836885397955
 -0.3258009107818365
  0.4761335071870796
 -0.2076850788308483
  0.5692231259136901
  0.9627746663812059
  0.009428632682882963
 -0.11719633849673108
 -0.8772739856669938
  0.16929144553007391
 -0.26149072780023486
  ⋮
  0.10653880994739139
  0.23915839789355517
 -0.08749999364363259
 -0.43512886247630145
 -0.6577800573029172
  0.033482459197247356
 -0.3229007365240325
  0.394966306821641
 -0.4678351244189623
 -0.2290972702487843
 -0.21348027521689217
  0.5448168434415765

In [125]:
xd = gmres(A, B, X0, max_iter)

200×1 Matrix{Float64}:
 -0.49055370584453417
 -0.41326485921491285
 -0.33194836885400925
 -0.32580091078187023
  0.4761335071870769
 -0.20768507883088155
  0.5692231259136918
  0.9627746663812592
  0.00942863268289637
 -0.11719633849676847
 -0.8772739856670171
  0.1692914455300838
 -0.26149072780023275
  ⋮
  0.10653880994737913
  0.23915839789356283
 -0.08749999364364724
 -0.43512886247631644
 -0.6577800573029214
  0.033482459197208825
 -0.32290073652402584
  0.39496630682165584
 -0.4678351244189779
 -0.2290972702487691
 -0.2134802752169016
  0.5448168434416039

In [129]:
x1 = bicgstab(A, B, X0, max_iter, accuracy)

NaN

In [127]:
#absolute error
abs_res = broadcast(abs, x2-x3)
abs_normal = broadcast(abs, xd-x3)

mean_abs_res = mean(abs_res)
mean_abs_normal = mean(abs_normal)

println("restarted:")
println(mean_abs_res)
println("base:")
println(mean_abs_normal)

restarted:
2.8495706228676145e-15
base:
1.6406135347826757e-14


In [128]:
#relative error (percentage)
rel_res = abs_res./x2 * 100
rel_normal = abs_normal./xd * 100

mean_rel_res = abs(mean(rel_res))
mean_rel_normal = abs(mean(rel_normal))

println("restarted:")
println(mean_rel_res)
println("base:")
println(mean_rel_normal)

restarted:
4.678287332097696e-13
base:
2.0531274930885808e-12
